In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
headers = {"User-Agent": "adit29my@gmail.com"} 

## Fetching historical data

In [5]:
def getCIKNumber(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

In [6]:
def getSubmissionData(ticker, headers=headers):
    cik = getCIKNumber(ticker)
    headers = headers
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()
    
    return pd.DataFrame(company_json["filings"]["recent"])

In [7]:
def getFilteredFilings(ticker, form, headers=headers):
    company_filings_df = getSubmissionData(
        ticker, headers=headers
    )
    if (form=='ten_k'):
        df = company_filings_df[company_filings_df["form"] == "10-K"]
    elif (form=='ten_q'):
        df = company_filings_df[company_filings_df["form"] == "10-Q"]
    
    df = df.set_index("reportDate")
    accession_df = df["accessionNumber"]
    return accession_df    

In [8]:
def getFacts(ticker, headers=headers):
    cik = getCIKNumber(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    company_facts = requests.get(url, headers=headers).json()
    return company_facts

In [9]:
def getFactsDF(ticker, headers=headers):
    facts = getFacts(ticker, headers)
    us_gaap_data = facts["facts"]["us-gaap"]
    df_data = []
    for fact, details in us_gaap_data.items():
        for unit in details["units"]:
            for item in details["units"][unit]:
                row = item.copy()
                row["fact"] = fact
                df_data.append(row)

    df = pd.DataFrame(df_data)
    df["end"] = pd.to_datetime(df["end"])
    df["start"] = pd.to_datetime(df["start"])
    df = df.drop_duplicates(subset=["fact", "end", "val"])
    df.set_index("end", inplace=True)
    labels_dict = {fact: details["label"] for fact, details in us_gaap_data.items()}
    return df, labels_dict

In [10]:
def getAnnualFacts(ticker, headers=headers):
    accession_nums = getFilteredFilings(ticker, form='ten_k')
    df, label_dict = getFactsDF(ticker, headers)
    ten_k = df[df["accn"].isin(accession_nums)]
    ten_k = ten_k[ten_k.index.isin(accession_nums.index)]
    pivot = ten_k.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [11]:
def getQuarterlyFacts(ticker, headers=headers):
    accession_nums = getFilteredFilings(ticker, form='ten_q')
    df, label_dict = getFactsDF(ticker, headers)
    ten_q = df[df["accn"].isin(accession_nums)]
    ten_q = ten_q[ten_q.index.isin(accession_nums.index)].reset_index(drop=False)
    ten_q = ten_q.drop_duplicates(subset=["fact", "end"], keep="last")
    pivot = ten_q.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [12]:
def getHistoricalData(ticker):
    quarterlyDF = getQuarterlyFacts(ticker)
    annualDF = getAnnualFacts(ticker)

    # Step 1: Find common rows
    common_indices = quarterlyDF.index.intersection(annualDF.index)

    # Step 2: Subset DataFrames to common rows
    Df1_common = quarterlyDF.loc[common_indices]
    Df2_common = annualDF.loc[common_indices]

    # Step 3: Combine the two DataFrames
    combined_df = pd.concat([Df1_common, Df2_common], axis=1)

    # Step 4: Sort the columns by date
    combined_df = combined_df.reindex(sorted(combined_df.columns), axis=1)

    return combined_df

## Financial Ratios

In [14]:
def addColumns(df, ratioDictionary):
    for col_name, (required_cols, operation) in ratioDictionary.items():
        if all(col in df.columns for col in required_cols):
            df[col_name] = operation(df)
        else:
            print(f"Skipping '{col_name}': Missing required columns {required_cols}.")
    return df

In [22]:
def calculateRatios(dataDf):
    
    df = pd.DataFrame(dataDf).T
    # df.index = pd.to_datetime(df.index)

    if 'Revenue, Net (Deprecated 2018-01-31)' in df.columns:
        df['Effective Revenue'] = df['Revenue, Net (Deprecated 2018-01-31)'].fillna(df['Revenue from Contract with Customer, Excluding Assessed Tax'])
    else:
        df['Effective Revenue'] = df['Revenue from Contract with Customer, Excluding Assessed Tax']
    
    df['Operating Margin Ratio'] = df['Operating Income (Loss)'] / df['Effective Revenue']
    ratioDictionary = {
        'Gross Margin Ratio': (['Gross Profit', 'Effective Revenue'], lambda df: df['Gross Profit'] / df['Effective Revenue']),
        'Operating Margin Ratio': (['Operating Income (Loss)', 'Effective Revenue'], lambda df: df['Operating Income (Loss)'] / df['Effective Revenue']),
        'Net Profit Margin Ratio': (['Net Income (Loss) Attributable to Parent', 'Effective Revenue'], lambda df: df['Net Income (Loss) Attributable to Parent'] / df['Effective Revenue']),
        'Return on Assets Ratio': (['Net Income (Loss) Attributable to Parent', 'Assets'], lambda df: df['Net Income (Loss) Attributable to Parent'] / df['Assets']),
        'Return on Equity Ratio': (['Net Income (Loss) Attributable to Parent', 'Stockholders\' Equity Attributable to Parent'], lambda df: df['Net Income (Loss) Attributable to Parent'] / df['Stockholders\' Equity Attributable to Parent']),
        'Current Ratio': (['Assets, Current', 'Liabilities, Current'], lambda df: df['Assets, Current'] / df['Liabilities, Current']),
        'Quick Ratio': (['Assets, Current', 'Inventory, Net', 'Liabilities, Current'], lambda df: df['Assets, Current'] - df['Inventory, Net'] / df['Liabilities, Current']),
        'Cash Ratio': (['Cash and Cash Equivalents, at Carrying Value', 'Liabilities, Current'], lambda df: df['Cash and Cash Equivalents, at Carrying Value'] / df['Liabilities']),
        'Debt to Equity (D/E) Ratio': (['Liabilities', 'Stockholders\' Equity Attributable to Parent'], lambda df: df['Liabilities'] / df['Stockholders\' Equity Attributable to Parent']),
        'Debt to Assets Ratio': (['Liabilities', 'Assets'], lambda df: df['Liabilities'] / df['Assets']),
        'Interest Coverage Ratio': (['Operating Income (Loss)', 'Interest Expense'], lambda df: df['Operating Income (Loss)'] / df['Interest Expense']),
        'Equity Ratio': (['Stockholders\' Equity Attributable to Parent', 'Assets'], lambda df: df['Stockholders\' Equity Attributable to Parent'] / df['Assets']),
        'Asset Turnover Ratio': (['Effective Revenue', 'Assets'], lambda df: df['Effective Revenue'] / df['Assets']),
        'Inventory Turnover Ratio': (['Cost of Goods and Services Sold', 'Inventory, Net'], lambda df: df['Cost of Goods and Services Sold'] / df['Inventory, Net']),
        'Receivables Turnover Ratio': (['Effective Revenue', 'Accounts Receivable, after Allowance for Credit Loss, Current'], lambda df: df['Effective Revenue'] / df['Accounts Receivable, after Allowance for Credit Loss, Current']),
        'Days Sales outstanding': (['Receivables Turnover Ratio'], lambda df: 365 / df['Receivables Turnover Ratio']),
        'Days Inventory outstanding': (['Inventory Turnover Ratio'], lambda df: 365 / df['Inventory Turnover Ratio']),
        'Payables Turnover Ratio': (['Cost of Goods and Services Sold', 'Accounts Payable, Current'], lambda df: df['Cost of Goods and Services Sold'] / df['Accounts Payable, Current']),
        'Operating Cash Flow Ratio': (['Net Cash Provided by (Used in) Operating Activities, Continuing Operations', 'Liabilities'], lambda df: df['Net Cash Provided by (Used in) Operating Activities, Continuing Operations'] / df['Liabilities']),
        'Capital Expenditure Coverage Ratio': (['Net Cash Provided by (Used in) Operating Activities, Continuing Operations', 'Payments to Acquire Property, Plant, and Equipment'], lambda df: df['Net Cash Provided by (Used in) Operating Activities, Continuing Operations'] / df['Payments to Acquire Property, Plant, and Equipment']),        
    }

    df_with_ratios = addColumns(df, ratioDictionary)
    df_with_ratios = df_with_ratios.applymap(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)

    return df_with_ratios.T

In [23]:
df = calculateRatios(appleData)
df

/var/folders/sy/sf3lnzz948s6nczk05xz5s640000gn/T/ipykernel_2444/115011178.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_with_ratios = df_with_ratios.applymap(lambda x: str(x) if isinstance(x, pd.Timestamp) else x)


end,2014-03-29,2014-06-28,2014-09-27,2014-12-27,2015-03-28,2015-06-27,2015-09-26,2015-12-26,2016-03-26,2016-06-25,...,2022-06-25,2022-09-24,2022-12-31,2023-04-01,2023-07-01,2023-09-30,2023-12-30,2024-03-30,2024-06-29,2024-09-28
fact,,,,,,,,,,,,,,,,,,,,,
"Accounts Payable, Current",1.891400e+10,2.053500e+10,3.019600e+10,3.800100e+10,2.315900e+10,2.647400e+10,3.549000e+10,3.331200e+10,2.509800e+10,2.631800e+10,...,4.834300e+10,6.411500e+10,5.791800e+10,4.294500e+10,4.669900e+10,6.261100e+10,5.814600e+10,4.575300e+10,4.757400e+10,6.896000e+10
"Accounts Receivable, after Allowance for Credit Loss, Current",9.700000e+09,1.078800e+10,1.746000e+10,1.670900e+10,1.090500e+10,1.037000e+10,1.684900e+10,1.295300e+10,1.222900e+10,1.171400e+10,...,2.180300e+10,2.818400e+10,2.375200e+10,1.793600e+10,1.954900e+10,2.950800e+10,2.319400e+10,2.183700e+10,2.279500e+10,3.341000e+10
"Accrued Income Taxes, Noncurrent",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.069900e+10,1.665700e+10,NaN,NaN,NaN,1.545700e+10,NaN,NaN,NaN,9.254000e+09
"Accrued Liabilities, Current",1.598400e+10,1.526400e+10,1.845300e+10,2.272400e+10,2.282700e+10,2.272400e+10,2.518100e+10,2.403200e+10,2.320800e+10,2.082000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Accumulated Depreciation, Depletion and Amortization, Property, Plant, and Equipment",1.528600e+10,1.686800e+10,1.839100e+10,2.035500e+10,2.230900e+10,2.439500e+10,2.678600e+10,2.904200e+10,3.084800e+10,3.254300e+10,...,7.151600e+10,7.234000e+10,6.804400e+10,6.966800e+10,7.078700e+10,7.088400e+10,7.251000e+10,7.169700e+10,7.262700e+10,7.344800e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Days Sales outstanding,3.429388e+01,2.799150e+01,5.666865e+01,NaN,3.001550e+01,2.077255e+01,4.312440e+01,6.231344e+01,8.828817e+01,2.533139e+01,...,9.592805e+01,2.608783e+01,7.400072e+01,6.903117e+01,8.723284e+01,2.810029e+01,7.079916e+01,8.782635e+01,9.699774e+01,3.118557e+01
Days Inventory outstanding,2.410141e+01,2.563378e+01,6.863787e+00,1.857629e+01,2.545672e+01,2.490743e+01,6.120288e+00,1.968393e+01,2.717603e+01,2.545768e+01,...,4.212612e+01,8.075698e+00,3.725270e+01,5.166345e+01,5.912028e+01,1.079129e+01,3.671995e+01,4.691803e+01,4.881288e+01,1.264257e+01
Payables Turnover Ratio,1.464471e+00,1.105284e+00,3.717645e+00,1.180443e+00,1.483397e+00,1.130317e+00,3.947281e+00,1.364343e+00,1.220655e+00,9.974922e-01,...,9.737501e-01,3.486641e+00,1.153735e+00,1.230877e+00,9.718409e-01,3.420118e+00,1.113060e+00,1.059646e+00,9.689957e-01,3.050348e+00


## Trends to plot

Labels from dataframe (Ratios):
1. Gross Margin Ratio
2. Operating Margin Ratio
3. Net Profit Margin Ratio
4. Return on Assets Ratio
5. Return on Equity Ratio
6. Earnings Per Share, Basic (already present in the facts, no need to calculate)
7. Earnings Per Share, Diluted (already present in the facts, no need to calculate)
8. Return on Investment (TO-DO)
9. Current Ratio
10. Quick Ratio
11. Cash Ratio
12. Debt to Equity (D/E) Ratio
13. Debt to Assets Ratio
14. Interest Coverage Ratio
15. Equity Ratio
16. Asset Turnover Ratio
17. Inventory Turnover Ratio
18. Receivables Turnover Ratio
19. Days Sales outstanding
20. Days Inventory outstanding
21. Payables Turnover Ratio
22. Cash Conversion Cycle - TODO
23. Price to Earnings (P/E) - TBD
24. Price to Book (P/B) - TBD
25. Price to sales (P/S) - TBD
26. Dividend Yield - TBD
27. Earnings Yield - TBD
28. Operating Cash Flow Ratio
29. Capital Expenditure Coverage Ratio




Miscellaneous trends:
1. Current Assets
2. Total Assets
3. Current Liabilities
4. Total Liabilities

## Get report filings

In [6]:
ticker = 'aapl'
cikNumber = getCIKNumber(ticker)
url = f"https://data.sec.gov/submissions/CIK{cikNumber}.json"
company_json = requests.get(url, headers=headers).json()
company_filings_df = pd.DataFrame(company_json["filings"]["recent"])

In [10]:
def getSubmissionData(ticker, headers=headers):
    cik = getCIKNumber(ticker)
    headers = headers
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()
    
    return pd.DataFrame(company_json["filings"]["recent"])

In [11]:
df = getSubmissionData('aapl')
df

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000320193-25-000002,2025-01-10,2025-01-01,2025-01-10T18:30:12.000Z,,3,,,,3,491830,0,0,xslF345X02/wk-form3_1736551805.xml,FORM 3
1,0001308179-25-000009,2025-01-10,,2025-01-10T16:37:12.000Z,34,DEFA14A,001-36743,25523516,,DEFA14A,181988,0,0,aapl4359751-defa14a.htm,DEFINITIVE ADDITIONAL PROXY SOLICITING MATERIALS
2,0001308179-25-000008,2025-01-10,2025-02-25,2025-01-10T16:31:18.000Z,34,DEF 14A,001-36743,25523460,,XBRL,32132682,1,1,aapl4359751-def14a.htm,DEFINITIVE PROXY STATEMENT
3,0001140361-25-000228,2025-01-03,2024-12-30,2025-01-03T16:30:56.000Z,34,8-K,001-36743,25506883,5.02,XBRL,263081,1,1,ef20040370_8k.htm,FORM 8-K
4,0000320193-24-000132,2024-12-18,2024-12-16,2024-12-18T18:30:20.000Z,,4,,,,4,11086,0,0,xslF345X05/wk-form4_1734564614.xml,FORM 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0001193125-14-160171,2014-04-25,,2014-04-25T17:30:48.000Z,33,S-8,333-195509,14786592,,S-8,157186,0,0,d715382ds8.htm,S-8
996,0001181431-14-016878,2014-04-24,2014-04-22,2014-04-24T18:34:41.000Z,,4,,,,4,8761,0,0,xslF345X03/rrd407684.xml,14.04.22 RICCIO FORM 4 - 10B5-1 SALE
997,0001193125-14-157311,2014-04-24,2014-03-29,2014-04-24T17:02:12.000Z,34,10-Q,000-10030,14782284,,10-Q,8315535,1,0,d694710d10q.htm,10-Q
998,0001193125-14-154883,2014-04-23,2014-04-23,2014-04-23T16:33:38.000Z,34,8-K,000-10030,14779103,"7.01,9.01",8-K,26214,0,0,d715379d8k.htm,FORM 8-K


In [14]:
def getFilteredFilings(
    ticker, form, headers=headers
):
    company_filings_df = getSubmissionData(
        ticker, headers=headers
    )
    df = company_filings_df[company_filings_df["form"] == form]    
    
    df = df.set_index("reportDate")
    accession_df = df["accessionNumber"]
    return accession_df

In [15]:
accession_df_ten_k = getFilteredFilings('aapl', "10-K")
accession_df_ten_k

reportDate
2024-09-28    0000320193-24-000123
2023-09-30    0000320193-23-000106
2022-09-24    0000320193-22-000108
2021-09-25    0000320193-21-000105
2020-09-26    0000320193-20-000096
2019-09-28    0000320193-19-000119
2018-09-29    0000320193-18-000145
2017-09-30    0000320193-17-000070
2016-09-24    0001628280-16-020309
2015-09-26    0001193125-15-356351
2014-09-27    0001193125-14-383437
Name: accessionNumber, dtype: object

In [16]:
accession_df_ten_q = getFilteredFilings('aapl', "10-Q")
accession_df_ten_q

reportDate
2024-06-29    0000320193-24-000081
2024-03-30    0000320193-24-000069
2023-12-30    0000320193-24-000006
2023-07-01    0000320193-23-000077
2023-04-01    0000320193-23-000064
2022-12-31    0000320193-23-000006
2022-06-25    0000320193-22-000070
2022-03-26    0000320193-22-000059
2021-12-25    0000320193-22-000007
2021-06-26    0000320193-21-000065
2021-03-27    0000320193-21-000056
2020-12-26    0000320193-21-000010
2020-06-27    0000320193-20-000062
2020-03-28    0000320193-20-000052
2019-12-28    0000320193-20-000010
2019-06-29    0000320193-19-000076
2019-03-30    0000320193-19-000066
2018-12-29    0000320193-19-000010
2018-06-30    0000320193-18-000100
2018-03-31    0000320193-18-000070
2017-12-30    0000320193-18-000007
2017-07-01    0000320193-17-000009
2017-04-01    0001628280-17-004790
2016-12-31    0001628280-17-000717
2016-06-25    0001628280-16-017809
2016-03-26    0001193125-16-559625
2015-12-26    0001193125-16-439878
2015-06-27    0001193125-15-259935
2015-03-2

In [17]:
def getAccessionNumbers(cikNumber):    

    url = f"https://data.sec.gov/submissions/CIK{cikNumber}.json"
    company_json = requests.get(url, headers=headers).json()

    company_filings_df = pd.DataFrame(company_json["filings"]["recent"])
    ten_k_filings = company_filings_df[company_filings_df["form"] == "10-K"]['accessionNumber'].tolist()
    ten_q_filings = company_filings_df[company_filings_df["form"] == "10-Q"]['accessionNumber'].tolist()

    ten_k_filings = [filing.replace('-', '') for filing in ten_k_filings]
    ten_q_filings = [filing.replace('-', '') for filing in ten_q_filings]

    return ten_k_filings, ten_q_filings

In [18]:
ticker = 'aapl'
cikNumber = getCIKNumber(ticker)
ten_k_filings, ten_q_filings = getAccessionNumbers(cikNumber)

In [20]:
def getFilingSummary(cikNumber, acc_num):
    session = requests.Session()    
    base_link = f"https://www.sec.gov/Archives/edgar/data/{cikNumber}/{acc_num}"
    filing_summary_link = f"{base_link}/FilingSummary.xml"
    filing_summary_response = session.get(
        filing_summary_link, headers=headers
    ).content.decode("utf-8")

    filing_summary_response = BeautifulSoup(filing_summary_response, "lxml-xml")

    return filing_summary_response

In [21]:
ten_k_filings[0]

'000032019324000123'

In [23]:
base_link = f"https://www.sec.gov/Archives/edgar/data/{cikNumber}/000032019324000123"
base_link

'https://www.sec.gov/Archives/edgar/data/0000320193/000032019324000123'

In [22]:
getFilingSummary(cikNumber, '000032019324000123')

<?xml version="1.0" encoding="utf-8"?>
<FilingSummary>
<Version>3.24.3</Version>
<ProcessingTime/>
<ReportFormat>html</ReportFormat>
<ContextCount>193</ContextCount>
<ElementCount>382</ElementCount>
<EntityCount>1</EntityCount>
<FootnotesReported>false</FootnotesReported>
<SegmentCount>73</SegmentCount>
<ScenarioCount>0</ScenarioCount>
<TuplesReported>false</TuplesReported>
<UnitCount>7</UnitCount>
<MyReports>
<Report instance="aapl-20240928.htm">
<IsDefault>false</IsDefault>
<HasEmbeddedReports>false</HasEmbeddedReports>
<HtmlFileName>R1.htm</HtmlFileName>
<LongName>0000001 - Document - Cover Page</LongName>
<ReportType>Sheet</ReportType>
<Role>http://www.apple.com/role/CoverPage</Role>
<ShortName>Cover Page</ShortName>
<MenuCategory>Cover</MenuCategory>
<Position>1</Position>
</Report>
<Report instance="aapl-20240928.htm">
<IsDefault>false</IsDefault>
<HasEmbeddedReports>false</HasEmbeddedReports>
<HtmlFileName>R2.htm</HtmlFileName>
<LongName>0000002 - Document - Auditor Information<

In [24]:
import requests

def get_latest_10k_url(cik: str):
    """
    Fetch the URL for the latest 10-K filing of a company using the SEC EDGAR API.

    Parameters:
    cik (str): The Central Index Key (CIK) of the company.

    Returns:
    str: URL of the latest 10-K filing or a message if no filing is found.
    """
    # Standardize CIK to 10 digits
    cik = cik.zfill(10)
    
    # SEC EDGAR API endpoint for company submissions
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
   
    
    try:
        # Fetch data from SEC API
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # Get filings from 'filings' key
        filings = data.get("filings", {}).get("recent", {})
        form_types = filings.get("form", [])
        accession_numbers = filings.get("accessionNumber", [])
        filing_dates = filings.get("filingDate", [])
        
        # Find the latest 10-K filing
        for form, acc_num, filing_date in zip(form_types, accession_numbers, filing_dates):
            if form == "10-K":
                # Construct filing URL
                filing_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num.replace('-', '')}/{acc_num}-index.html"
                return f"Latest 10-K Filing URL: {filing_url}\nFiling Date: {filing_date}"
        
        return "No 10-K filings found for this company."
    
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

# Example usage
cik = "320193"  # CIK for Apple Inc.
print(get_latest_10k_url(cik))

Latest 10-K Filing URL: https://www.sec.gov/Archives/edgar/data/0000320193/000032019324000123/0000320193-24-000123-index.html
Filing Date: 2024-11-01


In [25]:
import requests

def get_latest_10k_pdf_url(cik: str):
    """
    Fetch the URL for the latest 10-K filing PDF of a company using the SEC EDGAR API.

    Parameters:
    cik (str): The Central Index Key (CIK) of the company.

    Returns:
    str: URL of the latest 10-K filing PDF or a message if no filing is found.
    """
    # Standardize CIK to 10 digits
    cik = cik.zfill(10)
    
    # SEC EDGAR API endpoint for company submissions
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"    
    
    try:
        # Fetch data from SEC API
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # Get filings from 'filings' key
        filings = data.get("filings", {}).get("recent", {})
        form_types = filings.get("form", [])
        accession_numbers = filings.get("accessionNumber", [])
        primary_documents = filings.get("primaryDocument", [])
        
        # Find the latest 10-K filing
        for form, acc_num, primary_doc in zip(form_types, accession_numbers, primary_documents):
            if form == "10-K":
                # Construct filing PDF URL
                pdf_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num.replace('-', '')}/{primary_doc}"
                return f"Latest 10-K Filing PDF URL: {pdf_url}"
        
        return "No 10-K filings found for this company."
    
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

# Example usage
cik = "320193"  # CIK for Apple Inc.
print(get_latest_10k_pdf_url(cik))

Latest 10-K Filing PDF URL: https://www.sec.gov/Archives/edgar/data/0000320193/000032019324000123/aapl-20240928.htm


In [26]:
import requests

def get_latest_10q_pdf_url(cik: str):
    """
    Fetch the URL for the latest 10-Q filing PDF of a company using the SEC EDGAR API.

    Parameters:
    cik (str): The Central Index Key (CIK) of the company.

    Returns:
    str: URL of the latest 10-Q filing PDF or a message if no filing is found.
    """
    # Standardize CIK to 10 digits
    cik = cik.zfill(10)
    
    # SEC EDGAR API endpoint for company submissions
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"    
    
    try:
        # Fetch data from SEC API
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # Get filings from 'filings' key
        filings = data.get("filings", {}).get("recent", {})
        form_types = filings.get("form", [])
        accession_numbers = filings.get("accessionNumber", [])
        primary_documents = filings.get("primaryDocument", [])
        
        # Find the latest 10-Q filing
        for form, acc_num, primary_doc in zip(form_types, accession_numbers, primary_documents):
            if form == "10-Q":
                # Construct filing PDF URL
                pdf_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num.replace('-', '')}/{primary_doc}"
                return f"Latest 10-Q Filing PDF URL: {pdf_url}"
        
        return "No 10-Q filings found for this company."
    
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

# Example usage
cik = "320193"  # CIK for Apple Inc.
print(get_latest_10q_pdf_url(cik))

Latest 10-Q Filing PDF URL: https://www.sec.gov/Archives/edgar/data/0000320193/000032019324000081/aapl-20240629.htm


In [27]:
import requests

def get_latest_proxy_pdf_url(cik: str):
    """
    Fetch the URL for the latest proxy statement (DEF 14A) filing PDF of a company using the SEC EDGAR API.

    Parameters:
    cik (str): The Central Index Key (CIK) of the company.

    Returns:
    str: URL of the latest proxy statement PDF or a message if no filing is found.
    """
    # Standardize CIK to 10 digits
    cik = cik.zfill(10)
    
    # SEC EDGAR API endpoint for company submissions
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"    
    
    try:
        # Fetch data from SEC API
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # Get filings from 'filings' key
        filings = data.get("filings", {}).get("recent", {})
        form_types = filings.get("form", [])
        accession_numbers = filings.get("accessionNumber", [])
        primary_documents = filings.get("primaryDocument", [])
        
        # Find the latest DEF 14A filing
        for form, acc_num, primary_doc in zip(form_types, accession_numbers, primary_documents):
            if form == "DEF 14A":
                # Construct filing PDF URL
                pdf_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num.replace('-', '')}/{primary_doc}"
                return f"Latest Proxy Statement (DEF 14A) PDF URL: {pdf_url}"
        
        return "No proxy statement (DEF 14A) filings found for this company."
    
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

# Example usage
cik = "320193"  # CIK for Apple Inc.
print(get_latest_proxy_pdf_url(cik))

Latest Proxy Statement (DEF 14A) PDF URL: https://www.sec.gov/Archives/edgar/data/0000320193/000130817925000008/aapl4359751-def14a.htm
